In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [2]:
# model = ChatGroq(model_name="llama3-70b-8192", temperature=0)
model = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0)


In [3]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [5]:
# Constants
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

# Load, chunk and index the contents of the blog.
try:
    loader = WebBaseLoader(
        web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header")
            )
        ),
    )

    docs = loader.load()
except Exception as e:
    print(f"Error loading documents: {e}")
    # Handle error appropriately

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    """Join the page content of each document with two newlines."""
    return "\n\n".join(doc.page_content for doc in docs)

try:
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
except Exception as e:
    print(f"Error creating rag_chain: {e}")
    # Handle error appropriately

In [6]:
rag_chain.invoke("What is Task Decomposition?")

"Task Decomposition is a method to break down a complex task into smaller, manageable steps. It is used in autonomous agent systems, like those powered by Large Language Models (LLMs), to plan and execute complicated tasks. This can be done through simple prompting, task-specific instructions, or human inputs. The 'Chain of Thought' (CoT) technique is a standard prompting method that instructs the model to think step by step, transforming big tasks into multiple manageable ones."

In [6]:
# cleanup
vectorstore.delete_collection()